<a href="https://colab.research.google.com/github/DevRobertW/BibliotecaDev/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install flask flask-ngrok
!pip install pyngrok


In [4]:
from flask import Flask, render_template, request
import requests
import pandas as pd
from datetime import datetime
from flask_ngrok import run_with_ngrok
import os

app = Flask(__name__)
run_with_ngrok(app)

# Configurações
PORT = 8080

def fetch_data(data, empresa, token):
    url_viagens = f"http://servicos.cittati.com.br/WSIntegracaoCittati/Operacional/ConsultarViagens"
    params = {
        "data": data,
        "empresa": empresa
    }
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Basic {token}"
    }
    try:
        response = requests.get(url_viagens, params=params, headers=headers)
        response.raise_for_status()  # Lança um erro para códigos de status diferentes de 200
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados: {e}")
        return None

def process_data(data):
    if "viagens" in data:
        df = pd.DataFrame(data["viagens"])
        df = df[df['veiculo'] != "VNR"]
        df_filtered = df[(df['inicioRealizado'].isna()) | (df['fimRealizado'].isna())]
        df_sorted = df_filtered.sort_values(by='linha')
        return df_sorted
    else:
        return pd.DataFrame()

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        data = request.form.get("data")
        empresa = request.form.get("empresa")
        token = request.form.get("token")

        try:
            data = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")
        except ValueError:
            return "Formato de data inválido. Use o formato dd/mm/yyyy."

        response_data = fetch_data(data, empresa, token)

        if response_data:
            df_selecionado = process_data(response_data)
            if not df_selecionado.empty:
                return render_template("index.html", tables=[df_selecionado.to_html(classes='data', index=False)], titles=df_selecionado.columns.values)
            else:
                return "Nenhum dado encontrado após o processamento."
        else:
            return "Falha na obtenção de dados. Verifique suas credenciais ou tente novamente mais tarde."
    return render_template("index.html")

# Cria o diretório 'templates' se não existir
os.makedirs('templates', exist_ok=True)

# Cria o arquivo HTML
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Consulta de Viagens</title>
</head>
<body>
    <h1>Consulta de Viagens</h1>
    <form method="POST">
        <label for="data">Data (dd/mm/yyyy):</label>
        <input type="text" id="data" name="data" required>
        <br>
        <label for="empresa">Empresa:</label>
        <input type="text" id="empresa" name="empresa" required>
        <br>
        <label for="token">Token:</label>
        <input type="text" id="token" name="token" required>
        <br>
        <button type="submit">Consultar</button>
    </form>
    {% if tables %}
        <h2>Resultados</h2>
        <table>
            <tr>
            {% for title in titles %}
                <th>{{ title }}</th>
            {% endfor %}
            </tr>
            {% for row in tables %}
                <tr>{{ row|safe }}</tr>
            {% endfor %}
        </table>
    {% endif %}
</body>
</html>
"""

with open('templates/index.html', 'w') as f:
    f.write(template)

# Inicia o túnel ngrok (sem especificar o tipo de túnel)
from pyngrok import ngrok
public_url = ngrok.connect(8080)
print(f"Public URL: {public_url}")

# Executa a aplicação
if __name__ == "__main__":
    app.run()


ERROR:pyngrok.process.ngrok:t=2025-05-30T01:17:49+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The account "gps26@gevan.com.br" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n.

In [ ]:
from flask import Flask, render_template, request
import requests
import pandas as pd
from datetime import datetime
from flask_ngrok import run_with_ngrok
import os

app = Flask(__name__)
run_with_ngrok(app)

def fetch_data(data, empresa, token):
    url_viagens = f"http://servicos.cittati.com.br/WSIntegracaoCittati/Operacional/ConsultarViagens?data={data}&empresa={empresa}"
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Basic {token}"
    }
    response = requests.get(url_viagens, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def process_data(data):
    df = pd.DataFrame(data["viagens"])
    df = df[df['veiculo'] != "VNR"]
    df_filtered = df[(df['inicioRealizado'].isna()) | (df['fimRealizado'].isna())]
    df_sorted = df_filtered.sort_values(by='linha')
    return df_sorted

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        data = request.form.get("data")
        empresa = request.form.get("empresa")
        token = request.form.get("token")

        data = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")

        response_data = fetch_data(data, empresa, token)

        if response_data:
            df_selecionado = process_data(response_data)
            if not df_selecionado.empty:
                return render_template("index.html", tables=[df_selecionado.to_html(classes='data')], titles=df_selecionado.columns.values)
            else:
                return "Nenhum dado encontrado após o processamento."
        else:
            return "Falha na obtenção de dados."
    return render_template("index.html")

# Cria o diretório 'templates' se não existir
os.makedirs('templates', exist_ok=True)

# Cria o arquivo HTML
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Consulta de Viagens</title>
</head>
<body>
    <h1>Consulta de Viagens</h1>
    <form method="POST">
        <label for="data">Data (dd/mm/yyyy):</label>
        <input type="text" id="data" name="data" required>
        <br>
        <label for="empresa">Empresa:</label>
        <input type="text" id="empresa" name="empresa" required>
        <br>
        <label for="token">Token:</label>
        <input type="text" id="token" name="token" required>
        <br>
        <button type="submit">Consultar</button>
    </form>
    {% if tables %}
        <h2>Resultados</h2>
        {% for table in tables %}
            {{ table|safe }}
        {% endfor %}
    {% endif %}
</body>
</html>
"""

with open('templates/index.html', 'w') as f:
    f.write(template)

# Inicia o túnel ngrok (sem especificar o tipo de túnel)
from pyngrok import ngrok
public_url = ngrok.connect(8080)
print(f"Public URL: {public_url}")

# Executa a aplicação
app.run()

In [8]:
# Instalação dos pacotes necessários
!pip install flask flask-ngrok pyngrok

# Importação das bibliotecas
from flask import Flask, render_template, request
from pyngrok import ngrok
import pandas as pd
import requests
from datetime import datetime
import os
# Configuração do token do ngrok
ngrok.set_auth_token('2iu7X2ds0blGAPc1qFhMnL4fyTj_5QJoq1GZVUN1D2Fv4uurC')

# Criação do app Flask
app = Flask(__name__)

# Porta padrão
PORT = 5000
# Função para buscar dados
def fetch_data(data, empresa, token):
    url_viagens = f"http://servicos.cittati.com.br/WSIntegracaoCittati/Operacional/ConsultarViagens?data={data}&empresa={empresa}"
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Basic {token}"
    }
    response = requests.get(url_viagens, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Função para processar os dados
def process_data(data):
    df = pd.DataFrame(data["viagens"])
    df = df[df['veiculo'] != "VNR"]
    df_filtered = df[(df['inicioRealizado'].isna()) | (df['fimRealizado'].isna())]
    df_sorted = df_filtered.sort_values(by='linha')
    return df_sorted

# Rota principal
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        data = request.form.get("data")
        empresa = request.form.get("empresa")
        token = request.form.get("token")

        data = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")

        response_data = fetch_data(data, empresa, token)

        if response_data:
            df_selecionado = process_data(response_data)
            if not df_selecionado.empty:
                return render_template("index.html", tables=[df_selecionado.to_html(classes='data')], titles=df_selecionado.columns.values)
            else:
                return "Nenhum dado encontrado após o processamento."
        else:
            return "Falha na obtenção de dados."
    return render_template("index.html")

# Cria o diretório 'templates' se não existir
os.makedirs('templates', exist_ok=True)

# Cria o arquivo HTML
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Consulta de Viagens</title>
</head>
<body>
    <h1>Consulta de Viagens</h1>
    <form method="POST">
        <label for="data">Data (dd/mm/yyyy):</label>
        <input type="text" id="data" name="data" required>
        <br>
        <label for="empresa">Empresa:</label>
        <input type="text" id="empresa" name="empresa" required>
        <br>
        <label for="token">Token:</label>
        <input type="text" id="token" name="token" required>
        <br>
        <button type="submit">Consultar</button>
    </form>
    {% if tables %}
        <h2>Resultados</h2>
        {% for table in tables %}
            {{ table|safe }}
        {% endfor %}
    {% endif %}
</body>
</html>
"""

with open('templates/index.html', 'w') as f:
    f.write(template)

public_url = ngrok.connect(PORT)
print(f"Public URL: {public_url}")

# Rota principal
@app.route('/')
def home():
    return "Olá, esta é a aplicação Flask com ngrok!"

# Executa o app
app.run(port=PORT)


ERROR:pyngrok.process.ngrok:t=2025-05-30T01:22:03+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-30T01:22:03+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The account "gps26@gevan.com.br" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n.

In [9]:
# Instalação dos pacotes necessários
!pip install flask pyngrok pandas requests

# Importações
from flask import Flask, render_template, request
from pyngrok import ngrok
import pandas as pd
import requests
from datetime import datetime
import os

# Configuração opcional do token do ngrok (apenas 1x por máquina)
ngrok.set_auth_token('2iu7X2ds0blGAPc1qFhMnL4fyTj_5QJoq1GZVUN1D2Fv4uurC')

# Flask app
app = Flask(__name__)
PORT = 5000

# Função para buscar dados
def fetch_data(data, empresa, token):
    url_viagens = f"http://servicos.cittati.com.br/WSIntegracaoCittati/Operacional/ConsultarViagens?data={data}&empresa={empresa}"
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Basic {token}"
    }
    response = requests.get(url_viagens, headers=headers)
    return response.json() if response.status_code == 200 else None

# Processamento dos dados
def process_data(data):
    df = pd.DataFrame(data["viagens"])
    df = df[df['veiculo'] != "VNR"]
    df_filtered = df[(df['inicioRealizado'].isna()) | (df['fimRealizado'].isna())]
    return df_filtered.sort_values(by='linha')

# Rota principal
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        data = request.form.get("data")
        empresa = request.form.get("empresa")
        token = request.form.get("token")

        data = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")
        response_data = fetch_data(data, empresa, token)

        if response_data:
            df = process_data(response_data)
            if not df.empty:
                return render_template("index.html", tables=[df.to_html(classes='data')], titles=df.columns.values)
            else:
                return "Nenhum dado encontrado após o processamento."
        else:
            return "Falha na obtenção de dados."
    return render_template("index.html")

# Criação da pasta templates e HTML
os.makedirs('templates', exist_ok=True)
with open('templates/index.html', 'w') as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head><meta charset="UTF-8"><title>Consulta de Viagens</title></head>
<body>
    <h1>Consulta de Viagens</h1>
    <form method="POST">
        <label for="data">Data (dd/mm/yyyy):</label>
        <input type="text" id="data" name="data" required><br>
        <label for="empresa">Empresa:</label>
        <input type="text" id="empresa" name="empresa" required><br>
        <label for="token">Token:</label>
        <input type="text" id="token" name="token" required><br>
        <button type="submit">Consultar</button>
    </form>
    {% if tables %}
        <h2>Resultados</h2>
        {% for table in tables %}
            {{ table|safe }}
        {% endfor %}
    {% endif %}
</body>
</html>
""")

# Inicia túnel ngrok
public_url = ngrok.connect(PORT)
print(f"Public URL: {public_url}")

# Executa o app
app.run(port=PORT)


ERROR:pyngrok.process.ngrok:t=2025-05-30T01:23:12+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-30T01:23:12+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-30T01:23:12+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The account \"gps26@gevan.com.br\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The account "gps26@gevan.com.br" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n.